### **- MLflow**

Registrando o modelo no MLflow para rastreamento e versionamento

In [ ]:
!pip install mlflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 77.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.2/131.2 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 789.2/789.2 kB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 12.8 MB/s eta 0:00:00


In [ ]:
import pickle

# Carrega o arquivo com o modelo, threshold e features salvas
bundle_path = "/content/credit_risk_model_bundle.pkl"

with open(bundle_path, "rb") as f:
    artifacts = pickle.load(f)

model = artifacts["model"]
threshold = artifacts["threshold"]
feature_columns = artifacts["features"]

In [ ]:
import mlflow
import mlflow.sklearn

# Configura o MLflow
mlflow.set_tracking_uri("file:/content/mlruns")
mlflow.set_experiment("credit_risk_lightgbm")

/usr/local/lib/python3.12/dist-packages/mlflow/tracking/_tracking_service/utils.py:178: FutureWarning: The filesystem tracking backend (e.g., './mlruns') will be deprecated in February 2026. Consider transitioning to a database backend (e.g., 'sqlite:///mlflow.db') to take advantage of the latest MLflow features. See https://github.com/mlflow/mlflow/issues/18534 for more details and migration guidance. For migrating existing data, https://github.com/mlflow/mlflow-export-import can be used.
  return FileStore(store_uri, store_uri)
2026/02/02 22:42:15 INFO mlflow.tracking.fluent: Experiment with name 'credit_risk_lightgbm' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///content/mlruns/505811748746218022', creation_time=1770072135339, experiment_id='505811748746218022', last_update_time=1770072135339, lifecycle_stage='active', name='credit_risk_lightgbm', tags={}>

In [ ]:
import pickle

# Salva as features como artefato separado
features_path = "/content/feature_columns.pkl"

with open(features_path, "wb") as f:
    pickle.dump(feature_columns, f)


In [ ]:
with mlflow.start_run(run_name="lightgbm_credit_risk_v2"):

    # Parâmetros
    mlflow.log_param("model_type", "LightGBM")
    mlflow.log_param("threshold", threshold)
    mlflow.log_param("num_features", len(feature_columns))

    # Métricas (as finais do seu modelo)
    mlflow.log_metric("auc", 0.76)
    mlflow.log_metric("ks", 0.37)
    mlflow.log_metric("recall_class_1", 0.87)

    # Modelo
    mlflow.sklearn.log_model(
        model,
        name="model",
        registered_model_name="credit_risk_lightgbm"
    )

    # Artifacts
    mlflow.log_artifact(features_path)
    mlflow.log_artifact(bundle_path)

/usr/local/lib/python3.12/dist-packages/mlflow/models/model.py:1209: FutureWarning: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization.The recommended safe alternative is the 'skops' format.
  flavor.save_model(path=local_path, mlflow_model=mlflow_model, **kwargs)
/usr/local/lib/python3.12/dist-packages/mlflow/tracking/_model_registry/utils.py:216: FutureWarning: The filesystem model registry backend (e.g., './mlruns') will be deprecated in February 2026. Consider transitioning to a database backend (e.g., 'sqlite:///mlflow.db') to take advantage of the latest MLflow features. See https://github.com/mlflow/mlflow/issues/18534 for more details and migration guidance. For migrating existing data, https://github.com/mlflow/mlflow-export-import can be used.
  return FileStore(store_uri)
Successfully registered model 'credit_r

In [ ]:
from mlflow.tracking import MlflowClient

# Inicializa o cliente do MLflow
client = MlflowClient()

# Promove a versão do modelo para produção
client.transition_model_version_stage(
    name="credit_risk_lightgbm",
    version=1,
    stage="Production",
    archive_existing_versions=True
)

/tmp/ipython-input-1199160981.py:5: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1770072144778, current_stage='Production', deployment_job_state=None, description=None, last_updated_timestamp=1770072928984, metrics=[<Metric: dataset_digest=None, dataset_name=None, key='ks', model_id='m-e5aa16980556474f9287998143764c36', run_id='c3f149da5ecc4a13ada0756f84f44f95', step=0, timestamp=1770072136077, value=0.37>,
 <Metric: dataset_digest=None, dataset_name=None, key='recall_class_1', model_id='m-e5aa16980556474f9287998143764c36', run_id='c3f149da5ecc4a13ada0756f84f44f95', step=0, timestamp=1770072136078, value=0.87>,
 <Metric: dataset_digest=None, dataset_name=None, key='auc', model_id='m-e5aa16980556474f9287998143764c36', run_id='c3f149da5ecc4a13ada0756f84f44f95', step=0, timestamp=1770072136076, value=0.76>], model_id='m-e5aa16980556474f9287998143764c36', name='credit_risk_lightgbm', params={'model_type': 'LightGBM', 'num_features': '13', 'threshold': '0.12'}, run_id='c3f149da5ecc4a13ada0756f84f44f95', run_link=None, source

In [ ]:
# Consulta informações da versão do modelo no MLflow
client.get_model_version(
    name="credit_risk_lightgbm",
    version=1
)

<ModelVersion: aliases=[], creation_timestamp=1770072144778, current_stage='Production', deployment_job_state=None, description=None, last_updated_timestamp=1770072928984, metrics=[<Metric: dataset_digest=None, dataset_name=None, key='ks', model_id='m-e5aa16980556474f9287998143764c36', run_id='c3f149da5ecc4a13ada0756f84f44f95', step=0, timestamp=1770072136077, value=0.37>,
 <Metric: dataset_digest=None, dataset_name=None, key='recall_class_1', model_id='m-e5aa16980556474f9287998143764c36', run_id='c3f149da5ecc4a13ada0756f84f44f95', step=0, timestamp=1770072136078, value=0.87>,
 <Metric: dataset_digest=None, dataset_name=None, key='auc', model_id='m-e5aa16980556474f9287998143764c36', run_id='c3f149da5ecc4a13ada0756f84f44f95', step=0, timestamp=1770072136076, value=0.76>], model_id='m-e5aa16980556474f9287998143764c36', name='credit_risk_lightgbm', params={'model_type': 'LightGBM', 'num_features': '13', 'threshold': '0.12'}, run_id='c3f149da5ecc4a13ada0756f84f44f95', run_link=None, source

In [ ]:
# Carrega o modelo em produção a partir do MLflow Model Registry
model_prod = mlflow.pyfunc.load_model(
    "models:/credit_risk_lightgbm/Production"
)

import pandas as pd

# Cria um exemplo de entrada com todas as features zeradas
X_test_sample = pd.DataFrame(
    [ [0]*len(feature_columns) ],
    columns=feature_columns
)

# Realiza a predição usando o modelo em produção
model_prod.predict(X_test_sample)

array([0.82059225])

## **- Streamlit**

Vou usar o streamlit para testagem do modelo utilizando uma interface

In [ ]:
# Monta o schema padrão do dataset que foi usado para treino
%%writefile schema.py

model_features = [
    'EXT_SOURCE_1',
    'EXT_SOURCE_2',
    'EXT_SOURCE_3',
    'AMT_CREDIT',
    'AMT_ANNUITY',
    'PAYMENT_RATIO_MEAN',
    'POS_CNT_INSTALMENT_FUTURE_MEAN',
    'INST_NUM_INSTALMENT_NUMBER_COUNT',
    'PREV_CNT_PAYMENT_MEAN',
    'PREV_REFUSAL_RATE',
    'DAYS_EMPLOYED',
    'OWN_CAR_AGE',
    'CODE_GENDER'
]

categorical_features = ['CODE_GENDER']
numerical_features = [f for f in model_features if f not in categorical_features]


Writing schema.py


In [ ]:
%%writefile preprocessing.py

import pandas as pd
from schema import model_features, categorical_features

def validate_schema(df: pd.DataFrame):
    missing = set(model_features) - set(df.columns)
    if missing:
        raise ValueError(f'Colunas ausentes no dataset: {missing}')

def preprocess(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    # Validação
    validate_schema(df)

    # Selecionar apenas colunas esperadas
    df = df[model_features]

    # Tratamento categórico (regra congelada)
    if 'CODE_GENDER' in df.columns:
        df['CODE_GENDER'] = (
            df['CODE_GENDER']
            .astype(str)
            .map({'M': 1, 'F': 0})
            .fillna(-1)
        )

    # Garantir numéricos
    for col in df.columns:
        if col != 'CODE_GENDER':
            df[col] = pd.to_numeric(df[col], errors='coerce')

    # Tratamento de nulos
    df = df.fillna(0)

    return df


Writing preprocessing.py


In [ ]:
%%writefile app.py

import streamlit as st
import pandas as pd
import pickle
import plotly.express as px
import shap
import matplotlib.pyplot as plt

from preprocessing import preprocess

# ===============================
# Streamlit config
# ===============================
st.set_page_config(
    page_title="Modelo de Risco de Crédito",
    layout="wide"
)

# ===============================
# Load artifacts
# ===============================
with open("credit_risk_model_bundle.pkl", "rb") as f:
    artifacts = pickle.load(f)

model = artifacts["model"]
DEFAULT_THRESHOLD = float(artifacts["threshold"])

# ===============================
# Feature → Negócio
# ===============================
FEATURE_MAPPING = {
    "PAYMENT_RATIO_MEAN": "Comprometimento da renda influenciou a decisão de risco",
    "POS_CNT_INSTALMENT_FUTURE_MEAN": "Quantidade de parcelamentos futuros impactou o risco",
    "EXT_SOURCE_3": "Score externo teve influência relevante na avaliação do risco",
    "EXT_SOURCE_2": "Score externo contribuiu para a decisão de crédito",
    "EXT_SOURCE_1": "Comportamento capturado por score externo afetou o risco",
    "AMT_CREDIT": "Valor do crédito solicitado impactou a decisão",
    "AMT_ANNUITY": "Valor da parcela mensal influenciou o risco",
    "DAYS_EMPLOYED": "Histórico de vínculo empregatício impactou a decisão",
    "OWN_CAR_AGE": "Indicador patrimonial contribuiu para avaliação do risco",
    "INST_NULL_INSTALMENT_NUMBER_COUNT": (
        "Ausência de informações em contratos parcelados anteriores influenciou a decisão"
    ),
    "PREV_CNT_PAYMENT_MEAN": (
        "Comportamento médio de pagamentos em contratos anteriores impactou o risco"
    ),
    "PREV_REFUSAL_RATE": (
        "Histórico de recusas anteriores influenciou a avaliação do risco"
    ),
    "CODE_GENDER": (
        "Padrões estatísticos associados ao perfil demográfico influenciaram a decisão"
    )
}

def explicar_feature(feature, shap_value):
    base = FEATURE_MAPPING.get(
        feature,
        f"O comportamento da variável {feature} influenciou a decisão"
    )
    direcao = "aumentando o risco" if shap_value > 0 else "reduzindo o risco"
    return f"{base}, {direcao}."

# ===============================
# UI
# ===============================
st.title("Modelo de Risco de Crédito")
st.markdown("**Transformando risco em decisões claras e orientadas a lucro.**")
st.divider()

file = st.file_uploader("Envie um arquivo CSV para análise", type="csv")

threshold = st.slider(
    "Escolha o corte de risco",
    min_value=0.01,
    max_value=0.99,
    value=DEFAULT_THRESHOLD,
    step=0.01
)

# ===============================
# Sidebar - Negócio
# ===============================
st.sidebar.header("Parâmetros de Negócio")

ticket_medio = st.sidebar.number_input(
    "Ticket médio do crédito (R$)",
    min_value=1000,
    value=15000,
    step=1000
)

prejuizo_medio = st.sidebar.number_input(
    "Prejuízo médio por inadimplência (R$)",
    min_value=1000,
    value=12000,
    step=1000
)

# ===============================
# Upload e previsão
# ===============================
if file:
    df_raw = pd.read_csv(file)
    st.subheader("Prévia dos dados")
    st.dataframe(df_raw.head())

    if st.button("Executar Previsão"):
        df_proc = preprocess(df_raw)
        proba = model.predict(df_proc)

        df = df_raw.copy()
        df["PROB_DEFAULT"] = proba
        df["TARGET_PRED"] = (proba >= threshold).astype(int)
        df["DECISION"] = df["TARGET_PRED"].map({1: "NEGADO", 0: "APROVADO"})

        st.session_state["df"] = df
        st.session_state["df_proc"] = df_proc
        st.session_state["threshold"] = threshold

# ===============================
# Resultados
# ===============================
if "df" in st.session_state:

    df = st.session_state["df"]
    df_proc = st.session_state["df_proc"]

    st.subheader("Resultado da Previsão")
    st.dataframe(df)

    st.info(
        f"📌 Regra: probabilidade ≥ {st.session_state['threshold']:.2f} → NEGADO"
    )

    # ===============================
    # KPIs
    # ===============================
    aprovados = df[df["TARGET_PRED"] == 0]
    negados = df[df["TARGET_PRED"] == 1]
    total_clientes = len(df)

    st.subheader("📊 Visão Executiva")

    col1, col2, col3 = st.columns(3)
    col1.metric("Total de Clientes", total_clientes)
    col2.metric("Clientes Aprovados", len(aprovados))
    col3.metric("Clientes Negados", len(negados))

    receita_bruta = len(aprovados) * ticket_medio
    prejuizo_estimado = len(aprovados) * prejuizo_medio * 0.25
    lucro_liquido = receita_bruta - prejuizo_estimado

    col4, col5, col6 = st.columns(3)
    col4.metric("Receita Bruta (R$)", f"{receita_bruta:,.0f}")
    col5.metric("Prejuízo Estimado (R$)", f"{prejuizo_estimado:,.0f}")
    col6.metric("Lucro Líquido (R$)", f"{lucro_liquido:,.0f}")

    pie_df = pd.DataFrame({
        "Categoria": ["Lucro Líquido", "Prejuízo", "Receita Não Capturada"],
        "Valor (R$)": [
            max(lucro_liquido, 0),
            prejuizo_estimado,
            len(negados) * ticket_medio
        ]
    })

    fig_pie = px.pie(
    pie_df,
    names="Categoria",
    values="Valor (R$)",
    hole=0.45,
    title="Distribuição Financeira das Decisões",
    color="Categoria",
    color_discrete_map={
        "Receita Não Capturada": "#0b3c5d",  # azul escuro
        "Lucro Líquido": "#7fc97f",          # verde claro
        "Prejuízo": "#d62728"                # vermelho
    }
    )

    st.plotly_chart(fig_pie, use_container_width=True)

    # ===============================
    # KPI - Aprovação vs Negação por Gênero
    # ===============================
    st.subheader("⚖️ Aprovações e Negações por Gênero")

    if "CODE_GENDER" in df.columns:

        df_genero = df.copy()

        df_genero["GENERO"] = df_genero["CODE_GENDER"].map({
            "M": "Homem",
            "F": "Mulher"
        }).fillna("Não informado")

        kpi_genero = (
            df_genero
            .groupby(["GENERO", "DECISION"])
            .size()
            .reset_index(name="Quantidade")
        )

        fig_genero = px.bar(
            kpi_genero,
            x="GENERO",
            y="Quantidade",
            color="DECISION",
            barmode="group",
            text="Quantidade",
            title="Distribuição de Decisões por Gênero",
            color_discrete_map={
                "NEGADO": "#d62728",
                "APROVADO": "#7fc97f"
            }
            )

        fig_genero.update_layout(
            xaxis_title="Gênero",
            yaxis_title="Quantidade de Clientes",
            legend_title="Decisão",
            height=400
        )

        st.plotly_chart(fig_genero, use_container_width=True)

        st.caption(
            "⚠️ Este indicador é descritivo. Diferenças observadas não implicam, "
            "isoladamente, viés do modelo."
        )

    else:
        st.warning("A coluna CODE_GENDER não está presente no dataset.")


    # ===============================
    # EXPLICABILIDADE
    # ===============================
    st.subheader("🧠 Explicação da Decisão")

    explainer = shap.TreeExplainer(model)
    shap_values = explainer(df_proc)

    cliente_idx = st.number_input(
        "Escolha o índice do cliente",
        min_value=0,
        max_value=len(df_proc) - 1,
        step=1
    )

    shap_row = shap_values[cliente_idx]

    shap_df = pd.DataFrame({
        "feature": shap_row.feature_names,
        "shap_value": shap_row.values
    })

    TOP_N = 8

    top_reasons = (
        shap_df
        .assign(abs_shap=lambda x: x["shap_value"].abs())
        .sort_values("abs_shap", ascending=False)
        .head(TOP_N)
    )

    st.markdown("### 📌 Principais fatores que influenciaram a decisão")

    for _, row in top_reasons.iterrows():
        st.write(f"- **{explicar_feature(row['feature'], row['shap_value'])}**")

    # ===============================
    # CONCLUSÃO FINAL
    # ===============================
    decisao_cliente = df.loc[cliente_idx, "DECISION"]

    st.subheader("🧾 Conclusão da Análise")

    if decisao_cliente == "NEGADO":
        st.warning(
            "A solicitação foi **negada** porque a combinação dos principais fatores "
            "indicou **risco elevado de inadimplência**, especialmente relacionados a "
            "capacidade de pagamento, histórico de crédito e comportamento financeiro."
        )
    else:
        st.success(
            "A solicitação foi **aprovada** porque os principais indicadores apontam "
            "**capacidade de pagamento adequada**, histórico favorável e risco controlado "
            "segundo os critérios do modelo."
        )

    # ===============================
    # SHAP TÉCNICO
    # ===============================
    with st.expander("🔍 Detalhamento técnico (SHAP)"):
        fig, ax = plt.subplots()
        shap.plots.waterfall(
            shap_row,
            max_display=8,
            show=False
        )
        st.pyplot(fig, bbox_inches="tight")
        plt.close(fig)

    # ===============================
    # Explicação geral
    # ===============================
    with st.expander("📘 Como o modelo toma decisões"):
        st.markdown("""
        **1️⃣ O modelo estima o risco de inadimplência.**
        Utilizamos LightGBM com foco em reduzir perdas.

        **2️⃣ O threshold define a política de crédito.**
        Quanto menor, mais conservadora a decisão.

        **3️⃣ A decisão final é explicável e rastreável.**
        Cada decisão é acompanhada dos fatores mais relevantes.

        **4️⃣ SHAP garante transparência e auditabilidade.**
        """)


Writing app.py


In [ ]:
!pip install streamlit pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 27.4 MB/s eta 0:00:00
  Attempting uninstall: cachetools
    Found existing installation: cachetools 7.0.0
    Uninstalling cachetools-7.0.0:
      Successfully uninstalled cachetools-7.0.0


In [ ]:
# Executa a aplicação Streamlit em segundo plano
# Redireciona logs (stdout e stderr) para um arquivo
!streamlit run app.py &>/content/logs.txt &

In [ ]:
from pyngrok import ngrok

# Define o token de autenticação da sua conta ngrok (necessário para abrir túnel)
ngrok.set_auth_token("SEU_TOKEN_AQUI")


In [ ]:
from pyngrok import ngrok

# Cria um túnel público apontando para a porta 8501 (porta padrão do Streamlit)
ngrok.connect(8501)

<NgrokTunnel: "https://mercedez-unchagrined-indefectibly.ngrok-free.dev" -> "http://localhost:8501">